# CS109B - Milestone 3 

### To make our results more comparable accross models we updated the analyses with metadata as predictors to 20k dataset


Authors: Alessio Placitelli, Roberto Vitillo, Stephanie von Klot-Heydenfeldt

In [22]:
import itertools as it
import sys
import os
import operator
import math
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import pandas as pd
import tables

from pandas.io.json import json_normalize
from sklearn.decomposition import PCA
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.multiclass import OneVsRestClassifier
#from sklearn.model_selection import GridSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import zero_one_loss
%matplotlib inline
import sklearn.preprocessing as Preprocessing
from sklearn import preprocessing

## Update: use 20 K dataset

Import the data from the updated Milestone 2 datasets


Use the same data set size as for the images wich was 80% of 5000 images

In [23]:
X_train01= pd.read_csv("../data/X_train01.csv", sep='\t', encoding='utf-8').iloc[:4000,:]
X_test01= pd.read_csv("../data/X_test01.csv", sep='\t', encoding='utf-8').iloc[:1000,:]
Y_train01= pd.read_csv("../data/Y_train01.csv", sep='\t', encoding='utf-8').iloc[:4000,:]
Y_test01= pd.read_csv("../data/Y_test01.csv", sep='\t', encoding='utf-8').iloc[:1000,:]
y1_train01 =np.genfromtxt("../data/y1_train01.csv",delimiter=',')[:4000,:]
y1_test01 =np.genfromtxt("../data/y1_test01.csv",delimiter=',')[:1000,:]


In [24]:
GENRE_LIST = [
    'Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
    'Documentary', 'Drama', 'Family', 'Fantasy','Fiction', 'History',
    'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
    'TV Movie', 'Thriller', 'War', 'Western'
]


####  Check training data size

In [25]:
print "Number of observations, variables: ", X_train01.shape, y1_train01.shape
print "---------------"
#print "Type and number of non-missing observations for each variable"
#print "-"

#print X_train01.info()


Number of observations, variables:  (4000, 1498) (4000, 20)
---------------


#### For comparison of precision/recall/accuracy, by genre create randomly assigned prediciton based on test frequencies

In [26]:
def random_using_freqs(y):
    #assinging randomly a label to each of the genres
    rngenre = np.array(range( len(y)))
    np.random.seed(42)
    for i in range(y.shape[1]):
        rngenre_i = np.random.binomial(1, y[:,i].sum()*1.0/len(y), size=len(y))
        rngenre = np.vstack((rngenre, rngenre_i))

    rngenre = rngenre[1:rngenre.shape[1],:]

    random_genre = rngenre.reshape(rngenre.shape[1],rngenre.shape[0])
    return random_genre

In [27]:
y1_test01random = random_using_freqs(y1_test01)


In [28]:
def detailed_report(y_pred, test_y, confusion_matrix = False):
    print classification_report(test_y, y_pred,
                                target_names = GENRE_LIST)
    
    print("\n**** Per genre accuracy ****\n\n")

    per_genre_accuracies = []
    for i, label in enumerate(GENRE_LIST):
        per_genre_accuracies.append(accuracy_score(y_pred[:, i], test_y[:,i]))
        print("{} accuracy:\t{:.3f}".format(label, per_genre_accuracies[-1]))

    if confusion_matrix == True:
        print("\n****  ****\n\n")
        print("Per label confusion matrix")
        for i, label in enumerate( GENRE_LIST):
            print("\n****  ****\n\n")
            print "genre: ", GENRE_LIST[i]
            print pd.crosstab(y_pred[:,i],test_y[:,i], margins = True)
            #print()
            #print "recalls"
            #print pd.crosstab(y_pred[:,i],test_y[:,i]).apply(lambda r: r/r.sum(), axis=0)


    print("\n**** OTHER METRICS ****\n\n")
    print("Overall accuracy:\t{:.3f}".format(accuracy_score(test_y, y_pred)))
    print("Average per-genre accuracy:\t{:.3f}".format(np.mean(per_genre_accuracies)))
    print("Hamming loss:\t\t{:.3f}".format(hamming_loss(test_y, y_pred)))
    print("Zero one loss:\t\t{:.3f}".format(zero_one_loss(test_y, y_pred)))
    print("Jaccard similarity:\t{:.3f}".format(jaccard_similarity_score(test_y, y_pred)))

### linear SVM

In [8]:
def tune_svc(xtrain, ytrain, xtest, ytest, ytest_random):
    #http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html#sphx-glr-auto-examples-model-selection-grid-search-digits-py

    # Set the parameters by cross-validation
    model_to_set = OneVsRestClassifier(SVC(kernel="linear"))

    tuned_parameters = [
        {'estimator__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 50.0, 100.0, 1000.0], 'estimator__kernel': ['linear']} 
      #{'estimator__C': [0.001], 'estimator__kernel': ['linear']}
        #,
      #{'estimator__C': [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 50.0], 'estimator__gamma':  [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1, 0.2, 0.3], 'estimator__kernel': ['rbf']},
     ]


    #scores = ['precision', 'recall']
    scores = ['precision']
    
    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(model_to_set, 
                           param_grid = tuned_parameters, cv=3
                           #,scoring='%s_macro' % score, verbose = 5
                          )
        clf.fit(xtrain, ytrain)
        
        y_test_predict = clf.predict(xtest)
        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
            
    return clf.best_params_, classification_report(ytest,  y_test_predict), y_test_predict   



In [9]:
#print "============"
#print("Bag of words from keywords, tagline, overview plus actors and other features")


std_scale = preprocessing.StandardScaler().fit(np.array(X_train01.iloc[:,:-2]))
X_train_std = std_scale.transform(np.array(X_train01.iloc[:,:-2]))
X_test_std = std_scale.transform(np.array(X_test01.iloc[:,:-2]))

param_01b, class_report_01b, y1_test_predict_lsvm, = tune_svc(X_train_std, y1_train01, X_test_std, y1_test01, y1_test01random)


In [18]:
print "Final model parameters after CV"
#parameters (10-4 to 10**3 had resulted in 10-3 as best perdictor
#print param_01b

#rerun best model
clf_lsvm =  OneVsRestClassifier(SVC(kernel = "linear", C = 0.001))
clf_lsvm.fit(X_train_std, y1_train01)
y1_test_predict_lsvm = clf_lsvm.predict(X_test_std)

print("Detailed classification report:")
detailed_report(y1_test_predict_lsvm, y1_test01)
print("\n**** compare this result with random labeling ****\n\n")
print detailed_report(y1_test_predict_lsvm,  y1_test01random)

Detailed classification report:
                 precision    recall  f1-score   support

         Action       0.52      0.38      0.44       160
      Adventure       0.44      0.26      0.33        78
      Animation       0.08      0.75      0.15        12
         Comedy       0.70      0.40      0.51       277
          Crime       0.35      0.30      0.32        61
    Documentary       0.33      0.11      0.17         9
          Drama       0.69      0.72      0.70       494
         Family       0.19      0.41      0.26        34
        Fantasy       0.15      0.11      0.13        18
        Fiction       0.00      0.00      0.00         0
        History       0.12      0.28      0.17        18
         Horror       0.72      0.47      0.57        80
          Music       0.14      0.57      0.22         7
        Mystery       0.05      0.67      0.09        15
        Romance       0.38      0.31      0.34        74
Science Fiction       0.53      0.42      0.47        4

#### For presentation, find examples of well/badly predicted movies

In [19]:
TMDB_all =pd.read_pickle("../data/metadata_export_20k")
TMDB_clean = TMDB_all.drop_duplicates(['tmdb_id'])
TMDB_clean['release_year'] = pd.to_numeric(TMDB_clean['release_date'].str[:4])
TMDB_clean = TMDB_clean.dropna(subset = ["director", "overview", "actor2", "release_year"])
drop_variables = [i for i, item in enumerate(TMDB_all.isnull().mean()) if item > 0.2]
TMDB_clean = TMDB_clean.drop(TMDB_clean.columns[drop_variables], axis = 1)
TMDB_clean.reset_index(inplace = True)


In [121]:
print "5 well predicted movies"
z = 0
for i in range(len(y1_test_predict_lsvm)):
    if (y1_test_predict_lsvm[i]==y1_test01[i]).all() :
        if  sum(y1_test01[i])>2 and z<5:
            z = z+1
            print i, pd.DataFrame(y1_test_predict_lsvm[i], index = GENRE_LIST)
            print "\n"
            print X_test01.iloc[i,:][X_test01.iloc[i,:]>0]
            print "\n"
            print TMDB_clean.loc[X_test01['Unnamed: 0'][i]]
            print "\n ****************"
                
print "5 not so well predicted movies"
z = 0
for i in range(len(y1_test_predict_lsvm)):
    if (y1_test_predict_lsvm[i]<>y1_test01[i]).any() :
        
        if  sum(y1_test01[i])==1 and z<5:
            z = z+1
            if sum(y1_test_predict_lsvm[i])>2:
                print i, pd.DataFrame(y1_test_predict_lsvm[i], index = GENRE_LIST)
                print "\n"
                print X_test01.iloc[i,:][X_test01.iloc[i,:]>0]
                print "\n"
                print TMDB_clean.loc[X_test01['Unnamed: 0'][i]]
                print "\n ****************"

5 well predicted movies
434                  0
Action           1
Adventure        1
Animation        0
Comedy           0
Crime            0
Documentary      0
Drama            0
Family           0
Fantasy          0
Fiction          0
History          0
Horror           0
Music            0
Mystery          0
Romance          0
Science Fiction  0
TV Movie         0
Thriller         1
War              0
Western          0


Unnamed: 0                          2880
Sylvester Stallone                     1
action                                 1
air                                    1
armed                                  1
band                                   1
christmas                              1
city                                   1
disaster                               1
dog                                    1
driver                                 1
ends                                   1
escape                                 1
explosion                             

### SVM with RBF kernel

In [20]:
def tune_svc(xtrain, ytrain, xtest, ytest, ytest_random):
    #http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html#sphx-glr-auto-examples-model-selection-grid-search-digits-py

    # Set the parameters by cross-validation
    model_to_set = OneVsRestClassifier(SVC(kernel="linear"))

    tuned_parameters = [
     #   {'estimator__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 50.0, 100.0, 1000.0], 'estimator__kernel': ['linear']} 
      #{'estimator__C': [0.001], 'estimator__kernel': ['linear']}
        #,
      {'estimator__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 50.0], 'estimator__gamma':  [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1, 0.2, 0.3], 'estimator__kernel': ['rbf']},
     ]


    #scores = ['precision', 'recall']
    scores = ['precision']
    
    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(model_to_set, 
                           param_grid = tuned_parameters, cv=3,
                           #scoring='%s_macro' % score,    
                           verbose = 5
                          )
        clf.fit(xtrain, ytrain)
        
        y_test_predict = clf.predict(xtest)
        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
            
    return clf.best_params_, classification_report(ytest,  y_test_predict), y_test_predict   

# a search with higher parameters (10-3 to 10**3 resulted in 10-2 as best perdictor) for precision


print "============"
print("Bag of words from keywords, tagline, overview plus actors and other features")


std_scale = preprocessing.StandardScaler().fit(np.array(X_train01.iloc[:,:-2]))
X_train_std = std_scale.transform(np.array(X_train01.iloc[:,:-2]))
X_test_std = std_scale.transform(np.array(X_test01.iloc[:,:-2]))

param_01b, class_report_01b, y_test_pred_rbfsvm = tune_svc(X_train_std, y1_train01, X_test_std, y1_test01, y1_test01random)


In [32]:
print "Final model parameters after CV"
#print param_01b
print "C = 10, gamma = 0.0005"
#rerun best model
clf_rbfsvm =  OneVsRestClassifier(SVC(kernel = "rbf", C = 10, gamma = 0.0005))
clf_rbfsvm.fit(X_train_std, y1_train01)
y1_test_predict_rbfsvm = clf_rbfsvm.predict(X_test_std)

print("Detailed classification report:")
detailed_report(y1_test_predict_rbfsvm, y1_test01)
print("\n**** compare this result with random labeling ****\n\n")
print detailed_report(y1_test_predict_rbfsvm,  y1_test01random)

Final model parameters after CV
C = 10, gamma = 0.0005
Detailed classification report:
                 precision    recall  f1-score   support

         Action       0.50      0.34      0.40       160
      Adventure       0.38      0.21      0.27        78
      Animation       0.09      0.67      0.16        12
         Comedy       0.69      0.37      0.48       277
          Crime       0.35      0.18      0.24        61
    Documentary       0.33      0.44      0.38         9
          Drama       0.67      0.71      0.69       494
         Family       0.18      0.38      0.24        34
        Fantasy       0.15      0.11      0.13        18
        Fiction       0.00      0.00      0.00         0
        History       0.06      0.17      0.09        18
         Horror       0.80      0.44      0.56        80
          Music       0.09      0.29      0.13         7
        Mystery       0.05      0.67      0.09        15
        Romance       0.35      0.26      0.30        74


/home/steph/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


 ### Random forest 

In [21]:
from sklearn.ensemble import RandomForestClassifier

def tune_rf(xtrain, ytrain, xtest, ytest, ytest_random):
    #http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html#sphx-glr-auto-examples-model-selection-grid-search-digits-py

    # Set the parameters by cross-validation
    model_to_set = OneVsRestClassifier(RandomForestClassifier(random_state=0))

    tuned_parameters = [
        #10,12,14,16,18,20,
      {"estimator__max_depth": [22,30,50], 'estimator__n_estimators': [ 100,150,200, 500]}
     ]


    #scores = ['precision', 'recall']
    scores = ['precision']
    
    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(model_to_set, 
                           param_grid = tuned_parameters, cv=3,
                           #scoring='%s_macro' % score, 
                           verbose = 5)
        clf.fit(xtrain, ytrain)
        
        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()

        print("Detailed classification report:")
        
        y_test_predict = clf.predict(xtest)
        
        detailed_report(clf, xtest, ytest)
        print("\n**** compare this result with random labeling ****\n\n")

        print classification_report(ytest,  ytest_random, target_names =label_binarizer.classes_)
        print ()
    #returns only last in list "scores"    
    return clf.best_params_, classification_report(ytest,  y_test_predict), y_test_predict

    #with training set proportion=0.1 of all :    
    #best result {'estimator__max_depth': 22, 'estimator__n_estimators': 150}
#applying 3 different sampling methods

In [ ]:
print("All, sampling 0.1 of all, normalized")
param_01rf2, class_report_01rf2, y_test_pred_01rf2 = tune_rf(X_train_std, y1_train01, X_test_std, y1_test01, y1_test01random)

All, sampling 0.1 of all, normalized
# Tuning hyper-parameters for precision
()
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] estimator__max_depth=22, estimator__n_estimators=100 ............
[CV]  estimator__max_depth=22, estimator__n_estimators=100, score=0.005247 -  17.3s
[CV] estimator__max_depth=22, estimator__n_estimators=100 ............
[CV]  estimator__max_depth=22, estimator__n_estimators=100, score=0.022506 -  19.2s
[CV] estimator__max_depth=22, estimator__n_estimators=100 ............
[CV]  estimator__max_depth=22, estimator__n_estimators=100, score=0.013503 -  19.1s
[CV] estimator__max_depth=22, estimator__n_estimators=150 ............
[CV]  estimator__max_depth=22, estimator__n_estimators=150, score=0.005247 -  25.8s
[CV] estimator__max_depth=22, estimator__n_estimators=150 ............
[CV]  estimator__max_depth=22, estimator__n_estimators=150, score=0.025506 -  28.7s
[CV] estimator__max_depth=22, estimator__n_estimators=150 ............
[CV]  estimat

[Parallel(n_jobs=1)]: Done  17 tasks       | elapsed: 11.0min


[CV]  estimator__max_depth=30, estimator__n_estimators=150, score=0.021005 -  35.2s
[CV] estimator__max_depth=30, estimator__n_estimators=200 ............


In [123]:
print "Final model parameters after CV"
print "{estimator__max_depth': 50, 'estimator__n_estimators': 500}"
#print param_01rf2

#I have to rerun the full model after CV 
clf =  OneVsRestClassifier(RandomForestClassifier(max_depth= 50, n_estimators=500))
clf.fit(X_train_std, y1_train01)
y_test_pred_01rf2 = clf.predict(X_test_std)

print("Detailed classification report:")
detailed_report(y_test_pred_01rf2, y1_test01)
print("\n**** compare this result with random labeling ****\n\n")
print detailed_report(y_test_pred_01rf2,  y1_test01random)

Final model parameters after CV
{estimator__max_depth': 50, 'estimator__n_estimators': 500}
Detailed classification report:
                 precision    recall  f1-score   support

         Action       0.81      0.13      0.23       160
      Adventure       1.00      0.08      0.14        78
      Animation       0.19      0.67      0.29        12
         Comedy       0.93      0.15      0.26       277
          Crime       0.60      0.05      0.09        61
    Documentary       0.50      0.11      0.18         9
          Drama       0.68      0.76      0.71       494
         Family       0.36      0.15      0.21        34
        Fantasy       1.00      0.06      0.11        18
        Fiction       0.00      0.00      0.00         0
        History       0.29      0.28      0.29        18
         Horror       1.00      0.06      0.12        80
          Music       0.32      0.86      0.46         7
        Mystery       0.08      0.67      0.14        15
        Romance     

compare with a model that assignes zero to all

In [124]:
detailed_report(np.zeros(y1_test01.shape), y1_test01)

                 precision    recall  f1-score   support

         Action       0.00      0.00      0.00       160
      Adventure       0.00      0.00      0.00        78
      Animation       0.00      0.00      0.00        12
         Comedy       0.00      0.00      0.00       277
          Crime       0.00      0.00      0.00        61
    Documentary       0.00      0.00      0.00         9
          Drama       0.00      0.00      0.00       494
         Family       0.00      0.00      0.00        34
        Fantasy       0.00      0.00      0.00        18
        Fiction       0.00      0.00      0.00         0
        History       0.00      0.00      0.00        18
         Horror       0.00      0.00      0.00        80
          Music       0.00      0.00      0.00         7
        Mystery       0.00      0.00      0.00        15
        Romance       0.00      0.00      0.00        74
Science Fiction       0.00      0.00      0.00        43
       TV Movie       0.00    